In [1]:
!cat Dockerfile

# Copyright 2017-2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

# For more information on creating a Dockerfile
# https://docs.docker.com/compose/gettingstarted/#step-2-create-a-dockerfile
# https://github.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/pytorch_extending_our_containers/pytorch_extending_our_containers.ipynb
ARG REGION=us-east-1

# SageMaker PyTorch image
FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/pytorch-training:1.4.0-gpu-py3

RUN pip install fla

In [8]:
!./build_and_push.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  1.929MB
Step 1/8 : ARG REGION=us-east-1
Step 2/8 : FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/pytorch-training:1.4.0-gpu-py3
 ---> 8433beacd261
Step 3/8 : RUN pip install flair
 ---> Using cache
 ---> 0e481f0033fb
Step 4/8 : RUN pip install smart_open[gcp]
 ---> Using cache
 ---> 24145c15cdeb
Step 5/8 : ENV PATH="/opt/ml/co

In [9]:
# getting test data
import boto3
s3 = boto3.client('s3')
s3.download_file('sagemaker-us-east-1-953585160895',
                 'data/5ea9ec56ea2e7/preprocess_data.txt', 
                 'local_test/preprocess_data.txt')

In [7]:
from sagemaker import get_execution_role

role = get_execution_role()

import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

from sagemaker.estimator import Estimator

hyperparameters = {
    'epochs': 1,
    'column_names': '_id,message,image_concept,image,published,disabled',
    'target_names': 'published,disabled'    
}

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='flair_multilabels:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file://local_test')



# connect docker
# > docker run --rm -ti -v $(pwd)/local_test:/opt/ml/input/data/training flair_multilabels bash

# inside docker
# > cd /opt/ml/code
# > export SM_CHANNEL_TRAINING=/opt/ml/input/data/training/
# > export SM_MODEL_DIR=/opt/ml/
# > python multilabels.py --column_names _id,message,image_concept,image,published,disabled --epochs 1 --target_names published,disabled



Couldn't call 'get_role' to get Role ARN from role name SageMakerDev to get Role path.


Instance type = local_gpu


Using the short-lived AWS credentials found in session. They might expire while running.


Creating tmprjqazos5_algo-1-eqwxk_1 ... 
Attaching to tmprjqazos5_algo-1-eqwxk_12mdone
algo-1-eqwxk_1  | 2020-04-30 05:15:55,484 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-eqwxk_1  | 2020-04-30 05:15:55,486 sagemaker-containers INFO     Failed to parse hyperparameter column_names value _id,message,image_concept,image,published,disabled to Json.
algo-1-eqwxk_1  | Returning the value itself
algo-1-eqwxk_1  | 2020-04-30 05:15:55,486 sagemaker-containers INFO     Failed to parse hyperparameter target_names value published,disabled to Json.
algo-1-eqwxk_1  | Returning the value itself
algo-1-eqwxk_1  | 2020-04-30 05:15:55,511 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-eqwxk_1  | 2020-04-30 05:15:55,514 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-eqwxk_1  | 2020-04-30 05:15:55,516 sagemaker-containers INFO     Module default_user_module_name does not p

algo-1-eqwxk_1  | class: (published), train: 342, val: 156
algo-1-eqwxk_1  | class: (disabled), train: 1050, val: 441
algo-1-eqwxk_1  | 2020-04-30 05:16:01,243 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/twitter.gensim.vectors.npy not found in cache, downloading to /tmp/tmpvglma3bv
algo-1-eqwxk_1  | 2020-04-30 05:16:53,026 copying /tmp/tmpvglma3bv to cache at /root/.flair/embeddings/twitter.gensim.vectors.npy
algo-1-eqwxk_1  | 2020-04-30 05:16:53,492 removing temp file /tmp/tmpvglma3bv
algo-1-eqwxk_1  | 2020-04-30 05:16:54,235 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/twitter.gensim not found in cache, downloading to /tmp/tmpn4xkpy8w
algo-1-eqwxk_1  | 2020-04-30 05:17:03,296 copying /tmp/tmpn4xkpy8w to cache at /root/.flair/embeddings/twitter.gensim
algo-1-eqwxk_1  | 2020-04-30 05:17:03,365 removing temp file /tmp/tmpn4xkpy8w
algo-1-eqwxk_1  | TextClassifier(
algo-1-eqwxk_1  |   (document_embeddings): DocumentRNNEmbeddings(
algo-1-eqwxk

In [16]:
import sagemaker as sage

sess = sage.Session()

data_location = sess.upload_data('local_test', key_prefix='data/flair_test')
print(data_location)

s3://sagemaker-us-east-1-953585160895/data/flair_test


In [6]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'flair_multilabels'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

953585160895.dkr.ecr.us-east-1.amazonaws.com/flair_multilabels:latest


In [11]:
from sagemaker.estimator import Estimator

from sagemaker import get_execution_role

role = get_execution_role()

instance_type = 'ml.m4.xlarge'

estimator = Estimator(role=role,
                      base_job_name='flair-test',
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters)

estimator.fit(data_location)



Couldn't call 'get_role' to get Role ARN from role name SageMakerDev to get Role path.


2020-04-29 06:28:18 Starting - Starting the training job...
2020-04-29 06:28:21 Starting - Launching requested ML instances......
2020-04-29 06:29:25 Starting - Preparing the instances for training......
2020-04-29 06:30:34 Downloading - Downloading input data...
2020-04-29 06:30:57 Training - Downloading the training image...................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-29 06:34:39,962 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-29 06:34:39,963 sagemaker-containers INFO     Failed to parse hyperparameter column-names value _id,message,image_concept,published,disabled to Json.
Returning the value itself
2020-04-29 06:34:39,964 sagemaker-containers INFO     Failed to parse hyperparameter target-names value published,disabled to Json.
Returning the value itself
2020-04-29 06:34:39,966 sagemaker-containers INFO     No GPUs detected (normal if no gpus 


2020-04-29 06:34:38 Training - Training image download completed. Training in progress.2020-04-29 06:35:10,255 copying /tmp/tmpw860a7ej to cache at /root/.flair/embeddings/twitter.gensim.vectors.npy
2020-04-29 06:35:10,722 removing temp file /tmp/tmpw860a7ej
2020-04-29 06:35:11,580 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/twitter.gensim not found in cache, downloading to /tmp/tmpxl4cr_xi
2020-04-29 06:35:14,943 copying /tmp/tmpxl4cr_xi to cache at /root/.flair/embeddings/twitter.gensim
2020-04-29 06:35:15,019 removing temp file /tmp/tmpxl4cr_xi
TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('twitter')
    )
    (word_reprojection_map): Linear(in_features=100, out_features=128, bias=True)
    (rnn): GRU(128, 128, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=128, out_features=2, bias=True)
  (loss_function): BCEWi

> Epoch 0, val loss 0.03785077191096525, accuracy 0.5968447719944726, f1 0.5968396343790907
Saved model.
success!
2020-04-29 06:38:53,960 sagemaker-containers INFO     Reporting training SUCCESS

2020-04-29 06:39:48 Uploading - Uploading generated training model
2020-04-29 06:39:48 Completed - Training job completed
Training seconds: 554
Billable seconds: 554
